<div style="width: 100%; clear: both; font-family: Verdana;">
<div style="float: left; width: 50%;font-family: Verdana;">
<img src="https://github.com/Eligoze/att-rci-internal/blob/qa/RCI_DataAnalysis/eda/doc/att-logo1.png" align="left">
</div>
<div style="float: right; width: 200%;">
<p style="margin: 0; padding-top: 20px; text-align:right;color:rgb(193, 38, 184)"><strong>Axity - AT&T.
    Ciclo de vida de elementos de inventario</strong></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Exploratory Data Analysis


## Descripción

Analizaremos los datos de la fuente **ODK 12** que corresponde a los elementos que se encuentran en instalación de AT&T con un tratamiento estadístico descriptivo para la exploración, explotación y descubrimiento de los datos para un correcto tracking del ciclo de vida de los elementos de red. 

Primero cargamos las librerías necesarias.

#### Conectando al Datalake

In [1]:
import os
os.environ['JAVA_HOME'] = '/usr/java/jdk1.8.0_162'
os.environ['SPARK_HOME'] = '/opt/cloudera/parcels/CDH-6.2.0-1.cdh6.2.0.p0.967373/lib/spark'
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import HiveContext

In [2]:
conf = SparkConf().setAppName('Segregacion')  \
    .setMaster('yarn').set("spark.yarn.queue","root.eda")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
sqlContext = HiveContext(sc)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import re
import pandasql
from pyspark.sql.functions import udf ,col
from pyspark.sql.types import IntegerType,StringType

%matplotlib inline

from bokeh.io import show, output_notebook, output_file 
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20_11, Category20c_20, Category10_5,Category10_6, Category20_20, Plasma256
output_notebook()

Loading BokehJS ...

## 1. ODK 76
### 1. Recolección de los datos: 

Se crea el dataframe de spark con el universo de datos crudos.

In [4]:
df_load = spark.sql("SELECT * FROM default.tx_stg_06_tabular_odk_76").cache().toPandas()

Para las fuentes de los ODK's nos interesa conocer todos los elementos en sitio, por lo que haremos una limpieza en los campos que contengan características de los mismos. Creamos una funcion para el tratamiento del campo de sitio en spark el cual contiene ciertas reglas definidas para su limpieza.

Una muestra del ODK 76

In [5]:
df_load.head()

,id_form,clave_form,element_group,element,exist,TipoElemento_key,TipoElemento_value
0,179490,ALMCN,groupPedMovimiento-108,groupPedMovimiento-108,True,Codigo de sitio destino.,MEX-9221
1,179490,ALMCN,groupPedMovimiento-14,groupPedMovimiento-14,True,Codigo de sitio destino.,MEX-502H
2,179490,ALMCN,groupPedMovimiento-107,groupPedMovimiento-107,True,Pedido de Movimiento,63512719
3,179490,ALMCN,groupPedMovimiento-115,groupPedMovimiento-115,True,Pedido de Movimiento,63512811
4,179490,ALMCN,groupPedMovimiento-125,groupPedMovimiento-125,True,Pedido de Movimiento,63499475


### 2. Descripción de las fuentes.
En este apartado se hará una descripción a detalle de las fuentes para una mejor comprensión de los datos. Por cada fuente se mostrarán los tipos de datos, tamaño de la fuente, es decir, su dimensionalidad y una estadística descriptiva, en ese orden.

In [6]:
campos=df_load.columns
print 'Columnas de la fuente SOURCE son: ',list(campos)
pd.DataFrame(df_load.dtypes,columns=['Tipo de objeto SOURCE'])

Columnas de la fuente SOURCE son:  ['id_form', 'clave_form', 'element_group', 'element', 'exist', 'TipoElemento_key', 'TipoElemento_value']


,Tipo de objeto SOURCE
id_form,object
clave_form,object
element_group,object
element,object
exist,bool
TipoElemento_key,object
TipoElemento_value,object


Con la transformación que se llevo a cabo previamente, los campos **id_form, clave_form, element_group, element, exist** sirven para tener un control y mejor manejo del odk. Los campos **TipoElemento_key y TipoElemento_value** son los que se utilizarán para sacar indicadores.

A continuación se muestran los datos únicos en el campo **TipoElemento_Key**

In [7]:
df_load.TipoElemento_key.groupby(df_load['TipoElemento_key']).count()


TipoElemento_key
Codigo de sitio destino.    144
Pedido de Movimiento        144
QR                          144
TS Finalización               1
Name: TipoElemento_key, dtype: int64

Para el cálculo de indicadores que se va a realizar mas adelante, nos vamos a enfocar en el campo QR.

In [8]:
dfSerie2 =  df_load.TipoElemento_value.loc[(df_load.TipoElemento_key == 'QR')]

In [9]:
dfSerie2.describe(include='all')

count                        144
unique                       144
top       MEX6349895361050752EMB
freq                           1
Name: TipoElemento_value, dtype: object

Podemos observar que en la tabla tenemos el total del número de serie con el **count**, los campos únicos con **unique** y el valor que más se repite con el **top** y cuántas veces se retipe con el **freq**.

Podemos observar lo siguiente para los campos que contienen número de serie:

* Existen **144** registros con **QR**.
* **144** elementos con **QR** son únicos.
* El registro con **NMEX6349895361050752EMB** tiene una freuencia.

### Tamaño de la fuente

In [10]:
print('rows = ',df_load.shape[0],' columnas = ',df_load.shape[1])

('rows = ', 577, ' columnas = ', 7)


### 3. Exploración de los datos.

Se puede hace exploración de los datos para la fuente.

In [11]:
#Pasamos las columnas que queremos ver en nuestro describe:
NOrelevantes=['filedate', 'filename', 'hash_id', 'sourceid',
              'registry_state', 'datasetname', 'timestamp',
              'transaction_status', 'year', 'month', 'day']
relevantes=[v for v in df_load.columns if v not in NOrelevantes]

df_load[relevantes].describe(include='all')

,id_form,clave_form,element_group,element,exist,TipoElemento_key,TipoElemento_value
count,577,577,577,577,577,433,577
unique,1,1,145,146,1,4,433
top,179490,ALMCN,groupPedMovimiento-42,groupPallet-0,True,Codigo de sitio destino.,NA
freq,577,577,4,288,577,144,144


### 4. Calidad de los datos
En el parseo de nuestra fuente de ODK se creo el campo de *exist* que corresponde a la limpieza de los atributos que se encuentran en el formulario, con esto eliminando missing values.

### 5. Catalogos
Para estas fuentes no se encontraron catálogos.


### 6. Preparación de los datos


Se realiza una limpieza, es decir se remplazan los valores **NULL**,** **,**NA**, como **vacio**

In [12]:
df_load.replace(np.NaN,'vacio',inplace=True)
df_load.replace("NA",'vacio',inplace=True)
df_load.replace("NA",'vacio',inplace=True)
df_load.replace("null",'vacio',inplace=True)
df_load.replace("NULL",'vacio',inplace=True)

#pd.Series(['foo', 'fuz', np.nan]).str.replace('f.', np.NaN, regex=True,inplace=True)

### 7. Metricas KPI

Se mostrarán los KPIs generados. Se va a considerar el QR para hacer lo conteos ya que no tiene numero de serie.

#### Total de QR

In [13]:
Total_Elementos=df_load.loc[(df_load.TipoElemento_key=='QR') ].shape[0]
Total_Elementos

144

#### Total Elementos Trazables
Como no hay numero de serie, como tal no hay elementos trazables y solo son conteos.

#### Total Elementos NO Trazables
Igual como no existe trazabilidad, no se puede definir algun elemento trazable.

#### Total Elementos Trazables Únicos

In [14]:
Total_Tr_Unic=df_load.TipoElemento_value.loc[(df_load.TipoElemento_key=='QR') & (df_load.TipoElemento_value!='vacio') ].drop_duplicates().shape[0]
Total_Tr_Unic

144

#### Total de elementos trazables duplicados

In [15]:
Total_Tr_Dupli=Total_Tr_Unic
Total_Tr_Dupli

144

### Otros KPIS

En este apartado se muestra algunos de los hayazgos vistos en la exploración.

In [16]:
#Ajustar el df contra los kpis de la siguiente tabla:

KPIs=pd.DataFrame({'KPI':['Total Elementos'
                         ,'Total Trazables Unicos',
                         'Total Trazables Duplicados'],
                  'Resultado':[Total_Elementos,
                              Total_Tr_Unic,Total_Tr_Dupli]})

KPIs

,KPI,Resultado
0,Total Elementos,144
1,Total Trazables Unicos,144
2,Total Trazables Duplicados,144


In [17]:
#df_hive_kpi = spark.createDataFrame(KPIs)


In [18]:
#df_hive_kpi.write.mode("overwrite").saveAsTable("default.kpi_odk_76")

In [19]:
#sc.stop()